In [ ]:
To source Billboard data for capstone, weekly charts need to be scraped from 1/1/2010 - 12/31/2017.
example of first url: https://www.billboard.com/charts/hot-100/2010-01-09/

To do: 
1: create list of dates 
2: create list of URLs
3: develop scrape for one page
4: apply loop for all URLs in list (use .append so all data loads into a dataframe.  add a column with a "week of" identifier.

Charts are for the week prior: 

The Billboard Hot 100 is the music industry standard record chart in the United States for songs, published weekly by Billboard magazine. Chart rankings are based on sales (physical and digital), online streaming, and radio airplay in the U.S.[1]

A new chart is compiled and released online to the public by Billboard's website on Tuesdays but post-dated to the following Saturday, when the printed magazine first reaches newsstands.[2] The weekly tracking period for sales is currently Friday–Thursday, after being changed in July 2015. It was initially Monday–Sunday when Nielsen started tracking sales in 1991. This tracking period also applies to compiling online streaming data. Radio airplay is readily available on a real-time basis, unlike sales figures and streaming, but is also tracked on the same Friday–Thursday cycle, effective with the chart dated July 17, 2021.[3] Previously, radio was tracked Monday–Sunday and, before July 2015, Wednesday–Tuesday.[4]

In [2]:
import requests
from bs4 import BeautifulSoup as BS
import pandas as pd
from datetime import date, timedelta
import re

### Procure URLs for the Established Period

In [3]:
#url string for hot 100 chart; date is the suffix following the string below
base = 'https://www.billboard.com/charts/hot-100/'

In [4]:
#function - creates a date string for every 7 days from start date
def daterange(start_date, end_date):
    for n in range(0, int((end_date - start_date).days) +1, 7):
        yield start_date + timedelta(n)

#initiation - list
URL_list=[]

# intitiation - parameters
start = date(2010,1,9)
end = date(2017,12,30)

# append loop
for dt in daterange(start, end):
#    print(str(dt.strftime('%Y-%m-%d')))
     URL_list.append(base + str(dt.strftime('%Y-%m-%d')))

In [5]:
len(URL_list)

417

### Test URL; pull in first chart to query the page and build code

In [ ]:
# # test URL
# URL = 'https://www.billboard.com/charts/hot-100/2010-01-09'
# response = requests.get(URL)
# soup = BS(response.text)
# print(soup.prettify())

In [ ]:
## intiialize dataframe
billboard_data = pd.DataFrame()

In [ ]:
## establish for loop for URLs
for URL in URL_list:
    response = requests.get(URL)
    soup = BS(response.text)
    
    week_date = str(URL[-10:])
    
    week_of = []
    rank_current_week = []
    title = []
    artist = []
    rank_prior_week = []
    peak_pos = []
    weeks_on_chart = []

    content = soup.find_all("ul", attrs={"class":"o-chart-results-list-row"})
    for x in content:
        data_points = []
        song_title = x.select_one("h3").get_text().strip()
        chart_data = x.find_all("span")
        for point in chart_data:
            if point(text=re.compile('NEW')):
                chart_data.remove(point)
            elif point(text=re.compile('RE-\nENTRY')):
                chart_data.remove(point)
            else:
                point = point.get_text().strip()
                data_points.append(point)
        week_of.append(week_date)
        rank_current_week.append(data_points[0])
        title.append(song_title)
        artist.append(data_points[1])
        rank_prior_week.append(data_points[2])
        peak_pos.append(data_points[3])
        weeks_on_chart.append(data_points[4])
    
    chart_dict = ({
        'week_of':week_of, 
        'rank_current_week':rank_current_week, 
        'title':title, 'artist':artist, 
        'rank_prior_week':rank_prior_week, 
        'peak_pos':peak_pos, 
        'weeks_on_chart':weeks_on_chart})
    
    chart = pd.DataFrame(chart_dict)
    billboard_data = pd.concat([billboard_data, chart])

In [ ]:
billboard_data.info()

In [ ]:
billboard_data.to_csv('../data/Billboard/billboard_chart_data.csv', index=False)

In [ ]:
# ONE PAGE WORKING - DO NOT CHANGE
# week_date = str(URL[-11:-1])

# week_of = []
# rank_current_week = []
# title = []
# artist = []
# rank_prior_week = []
# peak_pos = []
# weeks_on_chart = []

# content = soup.find_all("ul", attrs={"class":"o-chart-results-list-row"})
# for x in content:
#     data_points = []
#     song_title = x.select_one("h3").get_text().strip()
#     chart_data = x.find_all("span")
#     for point in chart_data:
#         if point(text=re.compile('NEW')):
#             chart_data.remove(point)
#         elif point(text=re.compile('RE-\nENTRY')):
#             chart_data.remove(point)
#         else:
#             point = point.get_text().strip()
#             data_points.append(point)
#     week_of.append(week_date)
#     rank_current_week.append(data_points[0])
#     title.append(song_title)
#     artist.append(data_points[1])
#     rank_prior_week.append(data_points[2])
#     peak_pos.append(data_points[3])
#     weeks_on_chart.append(data_points[4])
    

In [ ]:
# chart_dict = {'week_of':week_of, 'rank_current_week':rank_current_week, 'title':title, 'artist':artist, 'rank_prior_week':rank_prior_week, 'peak_pos':peak_pos, 'weeks_on_chart':weeks_on_chart}

In [ ]:
#chart = pd.DataFrame(chart_dict)

In [ ]:
#chart.to_csv('../data/Billboard/test_chart.csv', index=False)